In [ ]:
### Examine Large Baskets
# A basket is a tuple (ID, Session, Scan, Date)

for df in df_all:
    for pid in df.ID.unique():
        for session in df.loc[df.ID == pid, 'Session'].unique():
            for scan in df.loc[(df.ID == pid) & (df.Session == session), 'Scan'].unique():
                for date in df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == scan), 'Date'].unique():
                    if df[(df.ID == pid) & (df.Session == session) & (df.Scan == scan) & (df.Date == date)].shape[0] >= 40:
                        print(f'({pid}, {session}, {scan}):', [str(d) for d in df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == scan), 'Date'].unique()])
    print()

In [ ]:
## Examine Missing Scan / ReceiptNum
for df in df_all:
    for pid in df.ID.unique():
        for session in df.loc[df.ID == pid, 'Session'].unique():
            for date in df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == -1), 'Date'].unique():
                print(f'({pid}, {session}, -1):', [str(d) for d in df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == -1), 'Date'].unique()])
    print()

In [ ]:
## Scan / ReceiptNum with multiple dates
for df in df_all:
    for pid in df.ID.unique():
        for session in df.loc[df.ID == pid, 'Session'].unique():
            for scan in df.loc[(df.ID == pid) & (df.Session == session), 'Scan'].unique():
                if (len(df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == scan), 'Date'].unique()) > 1) and (scan != -1):
                    print(f'({pid}, {session}, {scan}):', [str(d) for d in df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == scan), 'Date'].unique()])
    print()

In [ ]:
# TODO: explore evidence of systemic error
# Examine large baskets for multiple receipt error

In [ ]:
### Basket item verification across shared participants

## Build multi-index (ID, Basket, ItemNum) 
for df in df_shared:
    # Generate Basket column
    df['Basket'] = np.nan    
    for pid in df['ID'].unique():
        basket_counter = 0
        
        for session in df.loc[df['ID'] == pid, 'Session'].unique():
            
            for receipt in df.loc[(df['ID'] == pid) & (df['Session'] == session), 'Receipt'].unique():
                basket_counter += 1
                df.loc[(df['ID'] == pid) & (df['Session'] == session) & (df['Receipt'] == receipt), 'Basket'] = basket_counter
                # TODO interate on unique dates

    df['Basket'] = df['Basket'].astype('Int16')

    # Generate ItemNum column
    df['ItemNum'] = np.nan  
    for pid in df['ID'].unique():
        
        for basket in df.loc[df['ID'] == pid, 'BasketNum'].unique():
            df.loc[(df['ID'] == pid) & (df['BasketNum'] == basket), 'ItemNum'] = \
                    range(1, df.loc[(df['ID'] == pid) & (df['BasketNum'] == basket), 'ItemNum'].size + 1)

    df['ItemNum'] = df['ItemNum'].astype('Int16')
    df.drop(columns = ['Session', 'ReceiptNum'], inplace = True)

    # Create multi-index
    df.set_index(['ID', 'BasketNum', 'ItemNum'], inplace = True)
    df.sort_index(inplace = True)
    assert df.index.is_monotonic_increasing

In [ ]:
## Statistics on shared multi-indicies
assert df_shared[0].index.unique(0).equals(df_shared[1].index.unique(0)) and df_shared[1].index.unique(0).equals(df_shared[2].index.unique(0)) 

# Symmetric difference of three sets = (A delta B delta C) - (A & B & C)
# Use ^ for symmetric difference
index_sym_diff = list(set(df_shared[0].loc[shared_ids].index.symmetric_difference(df_shared[1].loc[shared_ids].index).symmetric_difference(df_shared[2].loc[shared_ids].index).tolist()) \
      - set(df_shared[0].loc[shared_ids].index.intersection(df_shared[1].loc[shared_ids].index).intersection(df_shared[2].loc[shared_ids].index).tolist()))
index_sym_diff = sorted(sorted(sorted(index_sym_diff, key = lambda i: i[2]), key = lambda i: i[1]), key = lambda i: i[0])
print('Symmetric difference of three indices\n', index_sym_diff)

In [ ]:
# TODO check for basket count agreement
for pid in df_shared[0].index.unique(0):
    print('\nID:', pid)
    for df in df_shared:
        print(df.loc[pid].index.unique(0)) # TODO restrict to PID

In [ ]:
# Drop unusable rows
essential_drop_count, unknown_drop_count, duplicate_drop_count, total_drop_count = [], [], [], []
for df in df_all:
    # missing essential values
    essential_drop_count.append(df[['ID', 'Session', 'ReceiptNum', 'Item']].isna().sum().sum())
    df.dropna(subset = ['ID', 'Session', 'ReceiptNum', 'Item'], inplace = True)
    
    # unreliable
    unknown_drop_count.append(df[df['Unknown'] == 'x'].isna().sum().sum())
    df.drop(df[df['Unknown'] == 'x'].index, inplace = True)
    
    # duplicate receipts
    duplicate_drop_count.append(df['Comment'].str.contains(r'duplicate|repeat').sum())
    df.drop(df[df['Comment'].str.contains(r'duplicate|repeat')].index, inplace = True)
    
    assert df[['ID', 'Session', 'ReceiptNum', 'Item']].notna().all(axis = None)
    assert df_max['Unknown'].isna().all()
    assert not df['Comment'].str.contains(r'duplicate|repeat').any()

print('Number of columns dropped due to missing essential columns:', essential_drop_count)
print('Number of columns dropped due to item being marked as "unknown":', unknown_drop_count)
print('Number of duplicate columns dropped:', duplicate_drop_count)

total_drop_count = [sum(c) for c in zip(essential_drop_count, unknown_drop_count, duplicate_drop_count)]
print('Total number of dropped rows:', total_drop_count)

drop_percentage = [round(c[0] / c[1] * 100) for c in zip(total_drop_count, df_sizes)]
print('Drop percentage:', drop_percentage)